In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np


2025-08-20 15:11:23.054512: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 15:11:23.059500: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 15:11:23.126993: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-20 15:11:24.776301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
## Load the Trained model,scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open("onehot_encoder_geo.pkl","rb") as file:
    label_encoder_geo=pickle.load(file)

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
## Example input data

input_data = {
'CreditScore': 600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 50000
}

In [4]:
one_encoded=label_encoder_geo.transform([[input_data["Geography"]]]).toarray()
one_encoded_df=pd.DataFrame(one_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
one_encoded_df

/home/amrinder/ANN Classification/myenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
## Converting key value painr into DataFrame
input_data=pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
#Combine one-hot encoded columns with input lines
input_data=pd.concat([input_data.drop('Geography',axis=1),one_encoded_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
## Encode categorical variables
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])
input_data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
##Scaling the data
input_scaled=scaler.transform(input_data)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [9]:
#Prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


array([[0.02133723]], dtype=float32)

In [10]:
prid_prob=prediction[0][0]
prid_prob

0.021337232

In [11]:
if prid_prob>0.5:
    print("Person is likely to churn.")
else:
    print("person is not likely to churn.")

person is not likely to churn.
